1. 生成测试证书
要实现 HTTPS，我们需要一个 SSL 证书 和 私钥，可以用 OpenSSL 生成（这一步在终端进行）
# 生成私钥
openssl genpkey -algorithm RSA -out private_key.pem

# 生成自签名证书（有效期365天）
openssl req -new -x509 -key private_key.pem -out certificate.pem -days 365


In [4]:
from OpenSSL import crypto
from datetime import datetime
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes


In [9]:
import socket
import ssl

# 服务器地址和端口
HOST = '127.0.0.1'
PORT = 8443  # HTTPS 通常用 443，这里用 8443 以避免权限问题

# 创建 TCP 套接字
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))  # 绑定 IP 和端口
server_socket.listen(5)  # 监听连接

# 使用 SSL 证书包装套接字
ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)  # 创建 TLS 服务器上下文
ssl_context.load_cert_chain(certfile="/Users/tangren/Documents/GitHub/openssl/Tianyu/certificate.pem", keyfile="/Users/tangren/Documents/GitHub/openssl/Tianyu/private_key.pem")  # 加载证书和私钥

# 让服务器支持 TLS
secure_socket = ssl_context.wrap_socket(server_socket, server_side=True)

print(f"[*] HTTPS 服务器运行中，访问 https://{HOST}:{PORT}/")

# 监听客户端连接
while True:
    client_socket, addr = secure_socket.accept()
    print(f"[+] 客户端 {addr} 连接成功")

    # 接收 HTTP 请求
    request = client_socket.recv(1024).decode()
    print("请求内容：\n", request)

    # 发送 HTTPS 响应
    response = "HTTP/1.1 200 OK\r\nContent-Type: text/plain\r\n\r\nHello, HTTPS!"
    client_socket.sendall(response.encode())

    # 关闭连接
    client_socket.close()


[*] HTTPS 服务器运行中，访问 https://127.0.0.1:8443/


SSLError: [SSL: SSLV3_ALERT_CERTIFICATE_UNKNOWN] ssl/tls alert certificate unknown (_ssl.c:1149)